In [1]:
import os
os.chdir("../..")

In [2]:
import torch
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import plot_training, plot_confusion_matrix

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

In [4]:
data_dir = "wildfire_dataset_scaled"
batch_size = 32
num_epochs = 10
learning_rate = 0.001
num_classes = 2

In [ ]:
print("[INFO] Loading augmented datasets...")
train_loader, val_loader, test_loader = load_datasets(data_dir, batch_size, augmentation="augmented")
print("[INFO] Augmented datasets loaded successfully!")



In [ ]:
print("\n[INFO] Starting Training for Model: vgg16 with Augmented Data\n")
print("[INFO] Initializing VGG16 model...")
model = initialize_model(
    model_name="densenet121",
    num_classes=num_classes,
    pretrained=True,
    freeze_all=True  # Augmented data: still freeze all layers
)
print("[INFO] Model initialized successfully!")

In [ ]:
print("[INFO] Setting up loss function and optimizer...")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
print("[INFO] Loss function and optimizer set up!")

In [ ]:
print("[INFO] Starting model training with augmented data...")
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=None,  # No learning rate scheduler for this stage
    num_epochs=num_epochs,
    device=device,
    save_path="outputs\models\augmented\densenet121.pth",
    early_stop_patience=5,  # Stop after 5 epochs without improvement
    monitor_metric="val_loss"
)
print("[INFO] Training completed successfully!")

In [ ]:
# Evaluate Model
print("[INFO] Evaluating the model...")
metrics = evaluate_model(
    model=model,
    test_loader=test_loader,
    classes=["No Fire", "Fire"],
    device=device
)
print(f"\n[INFO] Metrics for VGG16 with Augmented Data:\n{metrics}")

In [ ]:
# Save Plots
print("[INFO] Saving training and evaluation results...")
plot_training(history, "outputs/vgg16_augmented_training_curve.png")
plot_confusion_matrix(
    cm=metrics["confusion_matrix"],
    classes=["No Fire", "Fire"],
    output_path="outputs/vgg16_augmented_confusion_matrix.png"
)
print("[INFO] Results saved successfully!")